In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터셋 폴더 경로
dataset_dir = 'dataset'

# 클래스 이름
class_names = ['M', 'D']

# 각 클래스별 파일 경로
file_paths = {}
for class_name in class_names:
    file_paths[class_name] = [os.path.join(dataset_dir, class_name, file_name) for file_name in os.listdir(os.path.join(dataset_dir, class_name))]

# 각 클래스별 훈련 데이터와 검증 데이터 비율
train_ratio = 0.8
val_ratio = 0.2

# 훈련 데이터와 검증 데이터로 나누기
train_file_paths = {}
val_file_paths = {}
for class_name in class_names:
    train_files, val_files = train_test_split(file_paths[class_name], train_size=train_ratio, test_size=val_ratio, random_state=42)
    train_file_paths[class_name] = train_files
    val_file_paths[class_name] = val_files

# 훈련 데이터와 검증 데이터 폴더 경로
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'validation')

# 각 클래스별 훈련 데이터와 검증 데이터 폴더 생성
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)

# 각 클래스별 훈련 데이터와 검증 데이터 복사
for class_name in class_names:
    for file_path in train_file_paths[class_name]:
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, os.path.join(train_dir, class_name, file_name))
    for file_path in val_file_paths[class_name]:
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, os.path.join(val_dir, class_name, file_name))


In [21]:
import os
from PIL import Image

data_dir = "dataset"

# 하위 폴더 경로
subdirs = ['train/M', 'train/D', 'validation/M', 'validation/D']

# 이미지 크기 조정 함수
def resize_images(folder_path, new_size=(224, 224)):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img_resized = img.resize(new_size, resample=Image.LANCZOS)
            # 파일 확장자 변경
            new_filename = os.path.splitext(filename)[0] + ".jpg"
            new_img_path = os.path.join(folder_path, new_filename)
            img_resized.save(new_img_path)

# 각 하위 폴더 내 이미지 크기 변경 및 확장자 변경
for subdir in subdirs:
    folder_path = os.path.join(data_dir, subdir)
    resize_images(folder_path)


In [38]:
import os
from PIL import Image
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50

data_dir = "dataset"
subdirs = ['train/M', 'train/D', 'validation/M', 'validation/D']

def resize_images(folder_path, new_size=(224, 224)):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('RGB')
            img_resized = img.resize(new_size, resample=Image.LANCZOS)
 
            new_filename = os.path.splitext(filename)[0] + ".jpg"
            new_img_path = os.path.join(folder_path, new_filename)
            img_resized.save(new_img_path)

for subdir in subdirs:
    folder_path = os.path.join(data_dir, subdir)
    resize_images(folder_path)


In [40]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ResNet50 모델 불러오기
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 출력층 추가하기
x = resnet.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=resnet.input, outputs=predictions)

# 모델 컴파일하기
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 이미지 데이터 제너레이터 생성하기


train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('dataset/train', target_size=(224, 224), batch_size=32, class_mode='binary', color_mode='rgb')
val_generator = val_datagen.flow_from_directory('dataset/validation', target_size=(224, 224), batch_size=32, class_mode='binary', color_mode='rgb')

# 모델 학습하기
history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=val_generator, validation_steps=len(val_generator))

# 모델 검증하기
loss, accuracy = model.evaluate_generator(val_generator, steps=len(val_generator))
print('loss:', loss)
print('accuracy:', accuracy)


Found 340 images belonging to 2 classes.
Found 88 images belonging to 2 classes.


C:\Users\abc\AppData\Local\Temp\ipykernel_30144\4108045270.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=val_generator, validation_steps=len(val_generator))


Epoch 1/10
11/11 [==============================] - 108s 9s/step - loss: 3.8461 - accuracy: 0.4559 - val_loss: 182749.7031 - val_accuracy: 0.5000
Epoch 2/10
11/11 [==============================] - 83s 8s/step - loss: 1.7970 - accuracy: 0.5059 - val_loss: 252556.4375 - val_accuracy: 0.5000
Epoch 3/10
11/11 [==============================] - 78s 7s/step - loss: 1.5019 - accuracy: 0.5206 - val_loss: 114196.2266 - val_accuracy: 0.5000
Epoch 4/10
11/11 [==============================] - 77s 7s/step - loss: 1.2601 - accuracy: 0.5353 - val_loss: 21081.7617 - val_accuracy: 0.5000
Epoch 5/10
11/11 [==============================] - 78s 7s/step - loss: 0.8910 - accuracy: 0.5265 - val_loss: 1322.7891 - val_accuracy: 0.5000
Epoch 6/10
11/11 [==============================] - 78s 7s/step - loss: 1.1126 - accuracy: 0.4941 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 7/10
11/11 [==============================] - 78s 7s/step - loss: 0.8671 - accuracy: 0.5029 - val_loss: 0.6932 - val_accuracy: 0.50

C:\Users\abc\AppData\Local\Temp\ipykernel_30144\4108045270.py:31: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, accuracy = model.evaluate_generator(val_generator, steps=len(val_generator))


loss: 0.6931753158569336
accuracy: 0.5


In [49]:
from tensorflow.keras.applications.inception_v1 import InceptionV1
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# InceptionV1 모델 불러오기
inceptionv1 = InceptionV1(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 출력층 추가하기
x = inceptionv1.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inceptionv1.input, outputs=predictions)

# 모델 컴파일하기
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 이미지 데이터 제너레이터 생성하기
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('dataset/train', target_size=(224, 224), batch_size=32, class_mode='binary', color_mode='rgb')
val_generator = val_datagen.flow_from_directory('dataset/validation', target_size=(224, 224), batch_size=32, class_mode='binary', color_mode='rgb')

# 모델 학습하기
history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator), epochs=10, validation_data=val_generator, validation_steps=len(val_generator))

# 모델 검증하기
loss, accuracy = model.evaluate_generator(val_generator, steps=len(val_generator))
print('loss:', loss)
print('accuracy:', accuracy)


ModuleNotFoundError: No module named 'tensorflow.keras.applications.inception_v1'

In [48]:
import tensorflow as tf
print(tf.__version__)

2.10.0
